# HW1 布尔查询之BSBI与索引压缩

本次作业使用斯坦福大学[CS 276 / LING 286: Information Retrieval and Web Search](https://web.stanford.edu/class/cs276/)课程的代码框架来实现。具体来说主要包含的内容有：
1. [索引构建 (40%)](#索引构建与检索-(40%)) 使用BSBI方法模拟在内存不足的情况下的索引构建方式，并应用于布尔查询
2. [索引压缩 (30%)](#索引压缩-(30%)) 使用可变长编码对构建的索引进行压缩
3. [布尔检索 (10%)](#布尔联合检索-(10%)) 对空格分隔的单词查询进行联合（与）布尔检索
3. [实验报告 (10%)](#Report-(25%)) 描述你的代码并回答一些问题
4. [额外的编码方式 (10%)](#额外的编码方式-(10%)) 鼓励使用额外的编码方式对索引进行压缩 (例如, gamma-encoding)

In [99]:
# You can add additional imports here
import sys
import pickle as pkl
import array
import os
import io
import timeit
import contextlib

# 数据集

实验使用的文本数据是stanford.edu域下的网页内容，可从http://web.stanford.edu/class/cs276/pa/pa1-data.zip 下载。以下代码将大约170MB的文本数据下载到当前目录下，

In [100]:
import urllib.request
import zipfile

data_url = 'http://web.stanford.edu/class/cs276/pa/pa1-data.zip'
data_dir = 'pa1-data'
urllib.request.urlretrieve(data_url, data_dir+'.zip')
zip_ref = zipfile.ZipFile(data_dir+'.zip', 'r')
zip_ref.extractall()
zip_ref.close()

之后构建的索引会被存储到`output_dir`，`tmp`会存储测试数据（toy-data）所生成的一些临时文件

In [101]:
try: 
    os.mkdir('output_dir')
except FileExistsError:
    pass
try: 
    os.mkdir('tmp')
except FileExistsError:
    pass
try: 
    os.mkdir('toy_output_dir')
except FileExistsError:
    pass

在数据目录下有10个子目录（命名0-9）

In [102]:
sorted(os.listdir('pa1-data'))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

每一个子目录下的文件都包含一个独立网页的内容。可以认为在同一子目录下没有同名文件，即每个文件的绝对路径不会相同。

In [103]:
sorted(os.listdir('pa1-data/0'))[:10]

['3dradiology.stanford.edu_',
 '3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html',
 '3dradiology.stanford.edu_patient_care_case_studies.html',
 '5-sure.stanford.edu_',
 '50years.stanford.edu_',
 'a3cservices.stanford.edu_awards_nominate_',
 'a3cservices.stanford.edu_facilities_',
 'a3cservices.stanford.edu_lead_',
 'aa.stanford.edu_',
 'aa.stanford.edu_about_aviation.php']

所有的网页内容已经经过处理，仅包含由空格分隔开的单词，不再需要进行额外的标准化工作。

In [104]:
with open('pa1-data/0/3dradiology.stanford.edu_', 'r') as f:
    print(f.read())

3d radiology lab stanford university school of medicine stanford school of medicine 3d and quantitative imaging in the department of radiology search this site only stanford medical sites ways to give find a person alumni lane library ways to give find a person about us mission to develop and apply innovative techniques for efficient quantitative analysis and display of medical imaging data through interdisciplinary collaboration goals education to train physicians and technologists locally and worldwide in the latest developments in 3d and quantitative imaging research to develop new approaches to the exploration analysis and quantitative assesment of diagnostic images that result in a new and or more cost effective diagnostic approaches and b new techniques for the design and planning and monitoring of therapy patient care to deliver valid clinically relevant visualization and analysis of medical imaging data to the stanford community locations richard m lucas magnetic resonance imag

作业目录下有一个小的数据集文件夹`toy-data`。在使用完整网页数据集之前，我们会用它来测试我们的代码是否正确。

In [105]:
toy_dir = 'toy-data'

# 索引构建与检索 (40%)

作业的第一部分是使用**blocked sort-based indexing (BSBI)** 算法来构建倒排索引并实现布尔检索。关于BSBI算法可以参考老师课件或者斯坦福教材[Section 4.2](http://nlp.stanford.edu/IR-book/pdf/04const.pdf)。以下摘自教材内容

> To construct an index, we first make a pass through the collection assembling all term-docID pairs. We then sort the pairs with the term as the dominant key and docID as the secondary key. Finally, we organize the docIDs for each term into a postings list and compute statistics like term and document frequency. For small collections, all this can be done in memory. 

对于无法在内存一次性处理的较大数据集，将会使用到二级存储（如：磁盘）。

## IdMap

再次引用教材 Section 4.2:

> To make index construction more efficient, we represent terms as termIDs (instead of strings), where each termID is a unique serial number. We can build the mapping from terms to termIDs on the fly while we are processing the collection. Similarly, we also represent documents as docIDs (instead of strings).

我们首先定义一个辅助类`IdMap`，用于将字符串和数字ID进行相互映射，以满足我们在term和termID、doc和docID间转换的需求。

实现以下代码中的`_get_str` 和 `_get_id`函数，IdMap类的唯一接口是`__getitem__`，它是一个特殊函数，重写了下标运算`[]`,根据下标运算键的类型得到正确的映射值（如果不存在需要添加）。（特殊函数可参考[官方文档](https://docs.python.org/3.7/reference/datamodel.html#special-method-names)）
<br>
<br>
我们会用到字典来将字符串转换为数字，用列表来将数字转换为字符串。

In [106]:
class IdMap:
    """Helper class to store a mapping from strings to ids."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []
        
    def __len__(self):
        """Return number of terms stored in the IdMap"""
        return len(self.id_to_str)
        
    def _get_str(self, i):
        """Returns the string corresponding to a given id (`i`)."""
        ### Begin your code
        return self.id_to_str[i] #直接访问id_to_str数组的第i个元素
        ### End your code
        
    def _get_id(self, s):
        """Returns the id corresponding to a string (`s`). 
        If `s` is not in the IdMap yet, then assigns a new id and returns the new id.
        """
        ### Begin your code
        if s in self.str_to_id:      #若存在，直接访问并返回
            return self.str_to_id[s] 
        else:                        #不存在，需要在str_to_id和id_to_str中均添加此key-value对
            new_id=len(self.id_to_str)
            self.str_to_id[s]=new_id
            self.id_to_str.append(s)
            return new_id
        ### End your code
            
    def __getitem__(self, key):
        """If `key` is a integer, use _get_str; 
           If `key` is a string, use _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

确保代码能通过以下简单测试样例

In [107]:
testIdMap = IdMap()
assert testIdMap['a'] == 0, "Unable to add a new string to the IdMap"
assert testIdMap['bcd'] == 1, "Unable to add a new string to the IdMap"
assert testIdMap['a'] == 0, "Unable to retrieve the id of an existing string"
assert testIdMap[1] == 'bcd', "Unable to retrive the string corresponding to a\
                                given id"
try:
    testIdMap[2]
except IndexError as e:
    assert True, "Doesn't throw an IndexError for out of range numeric ids"
assert len(testIdMap) == 2

之后会需要你自己来写测试样例来确保你的程序正常运行

## 将倒排列表编码成字节数组

为了高效地从磁盘读写倒排列表（文档ID），我们将其存储为字节数组的形式。代码提供了`UncompressedPostings`类来用静态函数实现对倒排列表的编码和解码。在之后的任务中你需要使用该接口实现索引压缩版本（可变长编码）。

参考:
1. https://docs.python.org/3/library/array.html
2. https://pymotw.com/3/array/#module-array

In [108]:
class UncompressedPostings:
    
    @staticmethod
    def encode(postings_list):
        """Encodes postings_list into a stream of bytes
        
        Parameters
        ----------
        postings_list: List[int]
            List of docIDs (postings)
            
        Returns
        -------
        bytes
            bytearray representing integers in the postings_list
        """
        return array.array('L', postings_list).tobytes()
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes postings_list from a stream of bytes
        
        Parameters
        ----------
        encoded_postings_list: bytes
            bytearray representing encoded postings list as output by encode 
            function
            
        Returns
        -------
        List[int]
            Decoded list of docIDs from encoded_postings_list
        """
        
        decoded_postings_list = array.array('L')
        decoded_postings_list.frombytes(encoded_postings_list)
        return decoded_postings_list.tolist()

运行以下代码查看其工作方式

In [109]:
x = UncompressedPostings.encode([1,2,3])
print(x)
print(UncompressedPostings.decode(x))

b'\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00'
[1, 2, 3]


## 报告：
     普通编码方式是直接将int型转化为对应的字节数组，这样做不会压缩空间；
     但是通过字节流写文件会提高灵活性、兼容性

## 磁盘上的倒排索引

> With main memory insufficient, we need to use an external sorting algorithm, that is, one that uses disk. For acceptable speed, the central requirement of such an algorithm is that it minimize the number of random disk seeks during sorting - sequential disk reads are far faster than seeks. 
（zh-CN：磁盘上的倒排索引，当主存不足时，需要使用外部排序算法，即使用磁盘。对于可接受的速度，这种算法的核心要求是最小化随机磁盘访问次数，即顺序磁盘读取速度远远快于随机访问。）

在这一部分我们提供了一个基类`InvertedIndex`，之后会在此基础上构建它的子类`InvertedIndexWriter`, `InvertedIndexIterator` 和 `InvertedIndexMapper`。在Python中我们常用`cPickle`进行序列化，但是它并不支持部分读和部分写，无法满足BSBI算法的需要，所以我们需要定义自己的存储方式。

In [110]:
class InvertedIndex:
    """A class that implements efficient reads and writes of an inverted index 
    to disk
    
    Attributes
    ----------
    postings_dict: Dictionary mapping: termID->(start_position_in_index_file, 
                                                number_of_postings_in_list,
                                               length_in_bytes_of_postings_list)
        This is a dictionary that maps from termIDs to a 3-tuple of metadata 
        that is helpful in reading and writing the postings in the index file 
        to/from disk. This mapping is supposed to be kept in memory. 
        start_position_in_index_file is the position (in bytes) of the postings 
        list in the index file
        number_of_postings_in_list is the number of postings (docIDs) in the 
        postings list
        length_in_bytes_of_postings_list is the length of the byte 
        encoding of the postings list
    
    terms: List[int]
        A list of termIDs to remember the order in which terms and their   
        postings lists were added to index. 
        
        After Python 3.7 we technically no longer need it because a Python dict 
        is an OrderedDict, but since it is a relatively new feature, we still
        maintain backward compatibility with a list to keep track of order of 
        insertion. 
    """ 
       #postings_dict是termID->posting信息的列表
       #terms是termID的列表，用于记录插入的顺序
    def __init__(self, index_name, postings_encoding=None, directory=''):
        """
        Parameters
        ----------
        index_name (str): Name used to store files related to the index
        postings_encoding: A class implementing static methods for encoding and 
            decoding lists of integers. Default is None, which gets replaced
            with UncompressedPostings
        directory (str): Directory where the index files will be stored
        """
        #设置文件路径
        self.index_file_path = os.path.join(directory, index_name+'.index')
        self.metadata_file_path = os.path.join(directory, index_name+'.dict')
        #选择编码方式，默认是前面的UncompressedPostings，直至后面写了新的方法
        if postings_encoding is None: 
            self.postings_encoding = UncompressedPostings
        else:
            self.postings_encoding = postings_encoding
        self.directory = directory
        #初始化postings_dict和terms
        self.postings_dict = {}
        self.terms = []         #Need to keep track of the order in which the 
                                #terms were inserted. Would be unnecessary 
                                #from Python 3.7 onwards

    def __enter__(self):
        """Opens the index_file and loads metadata upon entering the context"""
        # Open the index file
        self.index_file = open(self.index_file_path, 'rb+')

        # Load the postings dict and terms from the metadata file
        with open(self.metadata_file_path, 'rb') as f:
            self.postings_dict, self.terms = pkl.load(f)
            self.term_iter = self.terms.__iter__()                       

        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        """Closes the index_file and saves metadata upon exiting the context"""
        # Close the index file
        self.index_file.close()
        
        # Write the postings dict and terms to the metadata file
        with open(self.metadata_file_path, 'wb') as f:
            pkl.dump([self.postings_dict, self.terms], f)

## 报告：
InvertedIndex是一个基础的类，上述三个函数定义了索引文件的打开、关闭方法，选择编码方式；并初始化postings_dict和terms两个字典/列表，这两个是index写入/读出的重要辅助信息。

因为是在与磁盘上的文件进行交互，所以我们提供了`__enter__`和`__exit__`函数，它使得我们能够像使用python中文件IO一样使用`with`语句。（参考[上下文管理器官方文档](https://docs.python.org/3/library/contextlib.html)）

以下是使用 `InvertedIndexWriter` 上下文管理器的实例:

```python
with InvertedIndexWriter('test', directory='tmp/') as index:
    # Some code here
```

## 索引

> BSBI (i) segments the collection into parts of equal size, (ii) sorts the termID-docID pairs of each part in memory, (iii) stores intermediate sorted results on disk, and (iv) merges all intermediate results into the final index. 

你需要将每一个子目录当做一个块（block），并且在构建索引的过程中每次只能加载一个块（模拟内存不足）。注意到我们是将操作系统意义上的块进行了抽象。你可以认为每个块足够小，能被装载进内存。

在这一部分，我们将阶段性地构造类`BSBIIndex`。函数`index`给出了BSBI算法的框架，而你的工作则是在接下来的部分中实现函数`parse_block`, `invert_write` 和 `merge`。

In [111]:
# Do not make any changes here, they will be overwritten while grading
class BSBIIndex:
    """ 
    Attributes
    ----------
    term_id_map(IdMap): For mapping terms to termIDs
    doc_id_map(IdMap): For mapping relative paths of documents (eg 
        0/3dradiology.stanford.edu_) to docIDs
    data_dir(str): Path to data
    output_dir(str): Path to output index files
    index_name(str): Name assigned to index
    postings_encoding: Encoding used for storing the postings.
        The default (None) implies UncompressedPostings
    """
    def __init__(self, data_dir, output_dir, index_name = "BSBI", 
                 postings_encoding = None):
        self.term_id_map = IdMap()
        self.doc_id_map = IdMap()
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.index_name = index_name
        self.postings_encoding = postings_encoding

        # Stores names of intermediate indices
        self.intermediate_indices = []
        
    def save(self):
        """Dumps doc_id_map and term_id_map into output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'wb') as f:
            pkl.dump(self.term_id_map, f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'wb') as f:
            pkl.dump(self.doc_id_map, f)
    
    def load(self):
        """Loads doc_id_map and term_id_map from output directory"""
        
        with open(os.path.join(self.output_dir, 'terms.dict'), 'rb') as f:
            self.term_id_map = pkl.load(f)
        with open(os.path.join(self.output_dir, 'docs.dict'), 'rb') as f:
            self.doc_id_map = pkl.load(f)
            
    def index(self):
        """Base indexing code
        
        This function loops through the data directories, 
        calls parse_block to parse the documents
        calls invert_write, which inverts each block and writes to a new index
        then saves the id maps and calls merge on the intermediate indices
        """
        for block_dir_relative in sorted(next(os.walk(self.data_dir))[1]):
            td_pairs = self.parse_block(block_dir_relative)
            index_id = 'index_'+block_dir_relative
            self.intermediate_indices.append(index_id)
            with InvertedIndexWriter(index_id, directory=self.output_dir, 
                                     postings_encoding=
                                     self.postings_encoding) as index:
                self.invert_write(td_pairs, index)
                td_pairs = None
        self.save()
        with InvertedIndexWriter(self.index_name, directory=self.output_dir, 
                                 postings_encoding=
                                 self.postings_encoding) as merged_index:
            with contextlib.ExitStack() as stack:
                indices = [stack.enter_context(
                    InvertedIndexIterator(index_id, 
                                          directory=self.output_dir, 
                                          postings_encoding=
                                          self.postings_encoding)) 
                 for index_id in self.intermediate_indices]
                self.merge(indices, merged_index)

## 报告：
（1）BSBI类中，index()函数搭建BSBI算法的整理框架。

（2）它首先调用parse_block()，从数据集目录下循环地依次处理每个子目录下的每个文件；

（3）然后对每个block调用invert_write(),构建索引并写入磁盘；

（4）最后调用merge_index()，将所有子目录的索引合并成一个索引文件。 这里使用contextlib管理器来自动管理多个文件的关闭。

### 解析

> The function `parse_block`  parses documents into termID-docID pairs and accumulates the pairs in memory until a block of a fixed size is full. We choose the block size to fit comfortably into memory to permit a fast in-memory sort. 

你需要将每一个子目录当做一个块，`parse_block`接收子目录路径作为参数。同一子目录下所有文件名都是不同的。

_注意 - 我们使用 `BSBIIndex` 继承 `BSBIIndex`, 这只是对一个已存在类添加新内容的简单方法。在这里只是用来切分类的定义（jupyter notebook内教学使用，无特殊含义）。_

In [112]:
class BSBIIndex(BSBIIndex):            
    def parse_block(self, block_dir_relative):
        """Parses a tokenized text file into termID-docID pairs
        
        Parameters
        ----------
        block_dir_relative : str
            Relative Path to the directory that contains the files for the block
        
        Returns
        -------
        List[Tuple[Int, Int]]
            Returns all the td_pairs extracted from the block
        
        Should use self.term_id_map and self.doc_id_map to get termIDs and docIDs.
        These persist across calls to parse_block
        """
        ### Begin your code
        td_pairs=[]
        #获取实际的block目录
        block_dir=os.path.join(self.data_dir,block_dir_relative)  
        #依次读取block目录下所有doc
        for filename in os.listdir(block_dir):
            #维护docID映射表
            doc=os.path.join(block_dir_relative,filename)#doc名称应该是文件夹/文件名，而不只是是文件名
            docID=self.doc_id_map.__getitem__(doc)
            #读取doc内容file_content
            file_path=os.path.join(block_dir,filename)
            if os.path.isfile(file_path):
                with open(file_path,'r',encoding='utf-8') as f:
                    file_content=f.read()
                    #遍历所有term，访问/生成termID,维护映射表
                    for term in file_content.split():
                        termID=self.term_id_map.__getitem__(term)
                        #生成td_pair
                        td_pair=(termID,docID)
                        td_pairs.append(td_pair)

        #对于td_pairs，只记出现内容，不计频率（去重）
        td_pairs=list(dict.fromkeys(td_pairs))
        return td_pairs
    
        ### End your code

观察函数在测试数据上是否正常运行

In [113]:
with open('toy-data/0/fine.txt', 'r') as f:
    print(f.read())
with open('toy-data/0/hello.txt', 'r') as f:
    print(f.read())

i'm fine , thank you

hi hi
how are you ?



In [114]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
BSBI_instance.parse_block('0')

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 1),
 (6, 1),
 (7, 1),
 (4, 1),
 (8, 1)]

In [115]:
BSBI_instance.parse_block('1')

[(9, 2), (10, 2), (11, 2), (4, 2), (2, 2), (12, 2), (13, 2), (8, 2), (12, 3)]

In [116]:
#测试：block[2]中的文件名称和block[0]中相同，但是可以识别为不同的doc
BSBI_instance.parse_block('2')

[(0, 4),
 (1, 4),
 (2, 4),
 (3, 4),
 (4, 4),
 (5, 5),
 (6, 5),
 (7, 5),
 (4, 5),
 (8, 5)]

写一些测试样例来确保`parse_block`方法正常运行（如：相同单词出现时是相同ID）

In [117]:
### Begin your code
#此测试样例测试toy-data下所有blocks，并检测相同单词出现是否是相同ID
#可以看到，'you'这个term对应的ID始终都是4
you_id1=BSBI_instance.term_id_map.__getitem__('you')
print(you_id1)

BSBI_instance.parse_block('1')
you_id2=BSBI_instance.term_id_map.__getitem__('you')
print(you_id2)

BSBI_instance.parse_block('2')
you_id3=BSBI_instance.term_id_map.__getitem__('you')
print(you_id3)

### End your code

4
4
4


## 报告：
（1）上述block_parse()函数的主要功能是：对一个block，遍历其目录下所有doc文件，对每个文件读取所有terms,并维护字典；所有termID-docID对返回；

（2）该方法只记录某term在某doc中出现，但不记录出现的次数和具体位置；

（3）经过上述测试样例（【55】-【58】），该方法可以正常工作；且对于重复的term、重复的doc，能够映射到同一ID；不同block之间ID映射关系是一贯相承的，不会冲突。


### 倒排表

> The block is then inverted and written to disk. Inversion involves two steps. First, we sort the termID-docID pairs. Next, we collect all termID-docID pairs with the same termID into a postings list, where a posting is simply a docID. The result, an inverted index for the block we have just read, is then written to disk.

在这一部分我们添加函数`invert_write`来实现由termID-docID对构建倒排表。 

但是，我们首先需要实现类`InvertedIndexWriter`。和列表类似，该类提供了append函数，但是倒排表不会存储在内存中而是直接写入到磁盘里。

In [118]:
class InvertedIndexWriter(InvertedIndex):
    """"""
    def __enter__(self):   
        self.index_file = open(self.index_file_path, 'wb+')         
        return self

    def append(self, term, postings_list):
        """Appends the term and postings_list to end of the index file.
        
        This function does three things, 
        1. Encodes the postings_list using self.postings_encoding
        2. Stores metadata in the form of self.terms and self.postings_dict
           Note that self.postings_dict maps termID to a 3 tuple of 
           (start_position_in_index_file, 
           number_of_postings_in_list, 
           length_in_bytes_of_postings_list)
        3. Appends the bytestream to the index file on disk

        Hint: You might find it helpful to read the Python I/O docs
        (https://docs.python.org/3/tutorial/inputoutput.html) for
        information about appending to the end of a file.
        
        Parameters
        ----------
        term:
            term or termID is the unique identifier for the term
        postings_list: List[Int]
            List of docIDs where the term appears
        """
        ### Begin your code
        # 编码postings_list
        postings_encoded=self.postings_encoding.encode(postings_list)

        # 存储元数据
        termID=term
        self.terms.append(termID)
  
        start_position_in_index_file=self.index_file.tell() #计算当前位置到文件末尾的距离 (3.1)
        number_of_posting_in_list=len(postings_list) #计算postings_list的长度 (3.2)
        length_in_bytes_of_postings_list=len(postings_encoded) #计算postings_encoded的长度 (3.3)
        self.postings_dict[termID]=(start_position_in_index_file,number_of_posting_in_list,length_in_bytes_of_postings_list)

        #写入磁盘
        self.index_file.write(postings_encoded)

        ### End your code

尽管还没有实现读取索引的类，我们还是可以用以下测试代码检测我们的实现。

In [119]:
with InvertedIndexWriter('test', directory='tmp/') as index:
    index.append(1, [2, 3, 4])
    index.append(2, [3, 4, 5])
    index.index_file.seek(0)
    assert index.terms == [1,2], "terms sequence incorrect"
    assert index.postings_dict == {1: (0, 3, len(UncompressedPostings.encode([2,3,4]))), 
                                   2: (len(UncompressedPostings.encode([2,3,4])), 3, 
                                       len(UncompressedPostings.encode([3,4,5])))}, "postings_dict incorrect"
    assert UncompressedPostings.decode(index.index_file.read()) == [2, 3, 4, 3, 4, 5], "postings on disk incorrect"

现在我们实现 `invert_write`，它将解析得到的td_pairs转换成倒排表，并使用`InvertedIndexWriter` 类将其写入磁盘。

In [120]:
class BSBIIndex(BSBIIndex):
    def invert_write(self, td_pairs, index):
        """Inverts td_pairs into postings_lists and writes them to the given index
        
        Parameters
        ----------
        td_pairs: List[Tuple[Int, Int]]
            List of termID-docID pairs
        index: InvertedIndexWriter
            Inverted index on disk corresponding to the block       
        """
        ### Begin your code
        #对termID-docID对排序
        td_pairs.sort(key=lambda x:(x[0],x[1]))#先按termID排序，再按docID排序
        postings_list=[]
        for i in range(len(td_pairs)):
            if i!=0 and td_pairs[i][0]!=td_pairs[i-1][0]:
                index.append(td_pairs[i-1][0],postings_list)#先将前面的写入index_file
                postings_list.clear()

            postings_list.append(td_pairs[i][1])
        index.append(td_pairs[-1][0],postings_list)#最后一个termID写入index_file
        return 
        ### End your code

我们可以在测试数据上读取一个块并观察倒排索引中包含的内容。
仿照`InvertedIndexWriter`部分写一些测试样例。

In [121]:
### Begin your code
#下述测试样例用到toy-data/0/的数据
with InvertedIndexWriter('test2', directory='tmp/') as index:
    #读取数据，构建索引，写入磁盘
    BSBI_instance2 = BSBIIndex(data_dir=toy_dir, output_dir = 'tmp/', index_name = 'toy')
    td_pairs=BSBI_instance2.parse_block('0')
    BSBI_instance2.invert_write(td_pairs,index)

    print(index.postings_dict)
    
    #检查
    index.index_file.seek(0)
    assert index.terms == [0,1,2,3,4,5,6,7,8], "terms sequence incorrect"
    assert index.postings_dict == {0:(0,1,4),
                                   1:(4,1,4),
                                   2:(8,1,4),
                                   3:(12,1,4),
                                   4:(16,2,8),
                                   5:(24,1,4),
                                   6:(28,1,4),
                                   7:(32,1,4),
                                   8:(36,1,4)}, "postings_dict incorrect"

    assert UncompressedPostings.decode(index.index_file.read()) == [0,0,0,0,0,1,1,1,1,1], "postings on disk incorrect"

### End your code

{0: (0, 1, 4), 1: (4, 1, 4), 2: (8, 1, 4), 3: (12, 1, 4), 4: (16, 2, 8), 5: (24, 1, 4), 6: (28, 1, 4), 7: (32, 1, 4), 8: (36, 1, 4)}


## 报告：
上述的invert_write()方法用于将parse_block()得到的td_pairs写入磁盘；
首先将td_pairs排序后合并相同termID得到对应的postings_list；
然后调用append()函数将每个termID-postings_list写入磁盘；
append()函数的将termID-postings_list写入磁盘，它维护元数据terms和postings_dict，并执行写入

### 合并
> The algorithm simultaneously merges the ten blocks into one large merged index. To do the merging, we open all block files simultaneously, and maintain small read buffers for the ten blocks we are reading and a write buffer for the final merged index we are writing. 

Python中的迭代模型非常自然地符合我们维护一个小的读缓存的要求。我们可以迭代地从磁盘上每次读取文件的一个倒排列表。我们通过构建`InvertedIndex`的子类`InvertedIndexIterator`来完成这个迭代任务。

In [122]:
class InvertedIndexIterator(InvertedIndex):
    """"""
    def __enter__(self):
        """Adds an initialization_hook to the __enter__ function of super class
        """
        super().__enter__()
        self._initialization_hook()
        return self

    def _initialization_hook(self):
        """Use this function to initialize the iterator
        """
        ### Begin your code
        self.index_file.seek(0)
        self=self.index_file.tell()
        ### End your code

    def __iter__(self): 
        return self
    
    def __next__(self):
        """Returns the next (term, postings_list) pair in the index.
        
        Note: This function should only read a small amount of data from the 
        index file. In particular, you should not try to maintain the full 
        index file in memory.
        """
        ### Begin your code
        #获取下一个(term, postings_list)存储在index_file中的postings_encoded的长度
        termID=next(self.term_iter)
        new_length=self.postings_dict[termID][2]
        #读取postings_encoded，解码
        postings_encoded=self.index_file.read(new_length)
        postings_list=self.postings_encoding.decode(postings_encoded)
        #组合成(term, postings_list)对返回
        return (termID, postings_list)

        ### End your code

    def delete_from_disk(self):
        """Marks the index for deletion upon exit. Useful for temporary indices
        """
        self.delete_upon_exit = True

    def __exit__(self, exception_type, exception_value, traceback):
        """Delete the index file upon exiting the context along with the
        functions of the super class __exit__ function"""
        self.index_file.close()
        if hasattr(self, 'delete_upon_exit') and self.delete_upon_exit:
            os.remove(self.index_file_path)
            os.remove(self.metadata_file_path)
        else:
            with open(self.metadata_file_path, 'wb') as f:
                pkl.dump([self.postings_dict, self.terms], f)

为了测试以上代码，我们先用`InvertedIndexWriter` 创建索引，然后再迭代遍历。写一些小的测试样例观察它是否正常运行。至少你得写一个测试，手工构建一个小的索引，用`InvertedIndexWriter`将他们写入磁盘，然后用`InvertedIndexIterator`迭代遍历倒排索引。

In [149]:
### Begin your code
#样例文本: doc1：I want to eat an apple     doc2: Do you like apple  doc3: I like to eat one 
#索引: （1,1),(2,1),(3,1),(4,1),(5,1),(6,1),(7,2),(8,2),(9,2),(6,2),(1,3),(9,3),(3,3),(4,3),(10,3)
#       I     (1,[1,3]);   want  (2,[1]);   to    (3,[1,3]);    eat   (4,[1,3]);  an    (5,[1]);  apple (6,[1,2])
#          Do    (7,[2]);     you   (8,[2]);   like  (9,[2,3]);     one   (10,[3])
with InvertedIndexWriter('test3', directory='tmp/') as index:
    index.append(1,[1,3])
    index.append(2,[1])
    index.append(3,[1,3])
    index.append(4,[1,3])
    index.append(5,[1])
    index.append(6,[1,2])
    index.append(7,[2])
    index.append(8,[2])
    index.append(9,[2,3])
    index.append(10,[3])
    #测试索引写入
    index.index_file.seek(0)
    assert index.terms == [1,2,3,4,5,6,7,8,9,10], "terms sequence incorrect"
    assert index.postings_encoding.decode(index.index_file.read()) == [1,3,1,1,3,1,3,1,1,2,2,2,2,3,3], "postings on disk incorrect"
    #测试迭代遍历
    IndexIterator=InvertedIndexIterator(index_name='test3')
    IndexIterator.__init__(index_name='test3',postings_encoding=None,directory='tmp/')
    IndexIterator.__enter__()

    true_indexs=[(1,[1,3]),(2,[1]),(3,[1,3]),(4,[1,3]),(5,[1]),(6,[1,2]),(7,[2]),(8,[2]),(9,[2,3]),(10,[3])]
    for i in range(len(true_indexs)):
        assert IndexIterator.__next__()==true_indexs[i],"index iterator incorrect"
    IndexIterator.__exit__(None,None,None)

### End your code

## 报告：
该InvertedIndexIterator类作为一个迭代器，本质上是从一个index_file中读取索引。它读取字节流，根据元数据的指示合并还原得到td_pairs，每次返回一个td_pair。

> During merging, in each iteration, we select the lowest termID that has not been processed yet using a priority queue or a similar data structure. All postings lists for this termID are read and merged, and the merged list is written back to disk. Each read buffer is refilled from its file when necessary.

我们将使用`InvertedIndexIterator`来完成读取的部分，用`InvertedIndexWriter`来将合并后的索引写入磁盘。

注意到我们之前一直使用`with` 语句来打开倒排索引文件，但是现在需要程序化地完成这项工作。可以看到我们在基类`BSBIIndex`的`index`函数中使用了`contextlib`([参考文档](https://docs.python.org/3/library/contextlib.html#contextlib.ExitStack))
你的任务是合并*打开的*`InvertedIndexIterator`对象（倒排列表），并且通过一个`InvertedIndexWriter`对象每次写入一个文档列表。

既然我们知道文档列表已经排过序了，那么我们可以在线性时间内对它们进行合并排序。事实上`heapq`([参考文档](https://docs.python.org/3.0/library/heapq.html)) 是Python中一个实现了堆排序算法的标准模板。另外它还包含一个实用的函数`heapq.merge`，可以将多个已排好序的输入（可迭代）合并成一个有序的输出并返回它的迭代器。它不仅可以用来合并倒排列表，也可以合并倒排索引词典。

为了让你快速上手`heapq.merge`函数，我们提供了一个简单的使用样例。给出两个以动物和树平均寿命排序的列表，我们希望合并它们。

In [124]:
import heapq
animal_lifespans = [('Giraffe', 28), 
                   ('Rhinoceros', 40), 
                   ('Indian Elephant', 70), 
                   ('Golden Eagle', 80), 
                   ('Box turtle', 123)]

tree_lifespans = [('Gray Birch', 50), 
                  ('Black Willow', 70), 
                  ('Basswood', 100),
                  ('Bald Cypress', 600)]

lifespan_lists = [animal_lifespans, tree_lifespans]

for merged_item in heapq.merge(*lifespan_lists, key=lambda x: x[1]):
    print(merged_item)

('Giraffe', 28)
('Rhinoceros', 40)
('Gray Birch', 50)
('Indian Elephant', 70)
('Black Willow', 70)
('Golden Eagle', 80)
('Basswood', 100)
('Box turtle', 123)
('Bald Cypress', 600)


注意使用`*`将`lifespan_lists`解包作为参数，并且使用`lambda`函数来给出进行排序的key。如果你对它们不熟悉可以参考文档([unpacking lists](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists), [lambda expressions](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions))。

In [125]:
import heapq
class BSBIIndex(BSBIIndex):
    def merge(self, indices, merged_index):
        """Merges multiple inverted indices into a single index
        
        Parameters
        ----------
        indices: List[InvertedIndexIterator]
            A list of InvertedIndexIterator objects, each representing an
            iterable inverted index for a block
        merged_index: InvertedIndexWriter
            An instance of InvertedIndexWriter object into which each merged 
            postings list is written out one at a time
        """
        ### Begin your code
        #建立总的排序列表
        total_postings=[]
        ind=0
        for iter in indices:#对每个indice，读取index_file的全部内容
            this_postings=[]
            for i in range(len(iter.terms)):
                this_postings.append(iter.__next__())
            if ind==0:#不同加入方式，保证得到[[],[],[]]的形式
                total_postings=[this_postings]
            else:
                total_postings.append(this_postings)
            ind+=1

        #按照termID排序
        merged_postings=list(heapq.merge(*total_postings, key=lambda x: x[0]))
        
        #对每个termID，合并其postings,排序+去重
        termID=merged_postings[0][0]
        docIDs=[merged_postings[0][1]]
        i=1
        while i <len(merged_postings):
            if merged_postings[i][0]==termID:#合并所有termID相同的docID
                docIDs.append(merged_postings[i][1])
                i+=1
            else:
                #对前面的termID进行处理
                merged_docIDs=list(heapq.merge(*docIDs))#合并+排序
                merged_docIDs=list(dict.fromkeys(merged_docIDs))#去重
                #print(termID,merged_docIDs)
                merged_index.append(termID,merged_docIDs)#写入总的index,同时维护其元数据

                #更新为新的termID
                termID=merged_postings[i][0]
                docIDs=[merged_postings[i][1]]
                i+=1
        #处理最后一个termID
        merged_docIDs=list(heapq.merge(*docIDs))#合并+排序
        merged_docIDs=list(dict.fromkeys(merged_docIDs))#去重
        #print(termID,merged_docIDs)
        merged_index.append(termID,merged_docIDs)#写入总的index

        return 
        ### End your code

## 报告：
合并：

该merge()方法，将多个index_file中的索引合并，并综合到merged_index文件中重新写入磁盘。该函数的思路框架如下：

（1）根据给定的indices迭代器，读取得到每个index_file中的td_pairs（分别作为this_posting）；total_postings是一个存储各个"this_posting"list的列表；

（2）在total_postings上应用heapq.merge()方法，对所有td_pairs按照termID排序,得到merged_postings;

（3）遍历merged_postings,将相邻的相同termID的所有docID整合到一起（也用heapq.merge()排序），并去重；得到该td_pair

（5）每个td_pair调用append函数写入merged_index文件中。

经过下述的测试，验证其功能正常、对于重复值、单个值等特殊情况都适用

首先确保它在测试数据上正常运行

In [126]:
BSBI_instance = BSBIIndex(data_dir=toy_dir, output_dir = 'toy_output_dir', )
BSBI_instance.index()
#经测试，可以实现排序、合并；且对于重复值、单个值都适用

接下来对整个数据集构建索引

In [127]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.index()

如果你在合并阶段出现了错误，可以利用以下代码来debug。

In [128]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.intermediate_indices = ['index_'+str(i) for i in range(10)]
with InvertedIndexWriter(BSBI_instance.index_name, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding) as merged_index:
    with contextlib.ExitStack() as stack:
        indices = [stack.enter_context(InvertedIndexIterator(index_id, directory=BSBI_instance.output_dir, postings_encoding=BSBI_instance.postings_encoding)) for index_id in BSBI_instance.intermediate_indices]
        BSBI_instance.merge(indices, merged_index)

## 布尔联合检索 (10%)

我们将实现BSBIIndex中的`retrieve`函数，对于给定的包含由空格分隔tokens的字符串查询，返回包含查询中所有tokens的文档列表。但是我们并不能迭代遍历整个索引或者将整个索引加载到内存来寻找相应的terms（索引太大）。

首先我们要实现`InvertedIndex`的子类`InvertedIndexMapper`，它能够找到对应terms在索引文件中位置并取出它的倒排记录表。

In [147]:
class InvertedIndexMapper(InvertedIndex):
    def __getitem__(self, key):
        return self._get_postings_list(key)
    
    def _get_postings_list(self, term):
        """Gets a postings list (of docIds) for `term`.
        
        This function should not iterate through the index file.
        I.e., it should only have to read the bytes from the index file
        corresponding to the postings list for the requested term.
        """
        ### Begin your code

        #先处理特殊情况：term不存在
        if term not in self.terms:
            print(term,"not in terms")
            return []
        else:
            #获取term对应posting_list在在文件的起始位置
            position=self.postings_dict[term][0]
            self.index_file.seek(position)
            length=self.postings_dict[term][2]
            #读取posting_list并解码
            posting_encoded=self.index_file.read(length)
            posting_list=self.postings_encoding.decode(posting_encoded)

        return posting_list
        ### End your code

写一些测试样例检测`_get_postings_list`的实现

In [130]:
### Begin your code
#下述测试样例借助test2已经存好的index_file；
#访问任何一个term，都可以从index_file中提取出正确的postings
true_answer={0:[0],
            1:[0],
            2:[0],
            3:[0],
            4:[0,1],
            5:[1],
            6:[1],
            7:[1],
            8:[1]}
#此样例对应于前面构建的test2
with InvertedIndexMapper(index_name='test2',postings_encoding=None,directory='tmp/') as indexmap:
    for i in range(9):
        real_answer=indexmap.__getitem__((i))
        print("real_answer:",real_answer)
        assert real_answer==true_answer[i],"wrong answer"

#查询一个不存在的term，返回空列表
notin_itemID=len(indexmap.terms)+3
print(indexmap.__getitem__(notin_itemID))
### End your code

real_answer: [0]
real_answer: [0]
real_answer: [0]
real_answer: [0]
real_answer: [0, 1]
real_answer: [1]
real_answer: [1]
real_answer: [1]
real_answer: [1]
12 not in terms
[]


现在我们获得了查询中terms对应的倒排记录表，接着需要求他们的交集。这部分与我们之前作业的merge方法类似，请实现`sorted_intersect`函数，遍历两个有序列表并在线性时间内合并。

In [131]:
import heapq
def sorted_intersect(list1, list2):
    """Intersects two (ascending) sorted lists and returns the sorted result
    
    Parameters
    ----------
    list1: List[Comparable]
    list2: List[Comparable]
        Sorted lists to be intersected
        
    Returns
    -------
    List[Comparable]
        Sorted intersection        
    """
    ### Begin your code
    #合并+排序
    total_list= [list1, list2]
    merged_list=[]
    for num in heapq.merge(*total_list, key=lambda x: x):
        merged_list.append(num)
    #筛选都有的-->交集
    intersection=[]
    i=0
    while i < len(merged_list):
        if i!=len(merged_list)-1 and merged_list[i]==merged_list[i+1]:
            intersection.append(merged_list[i])
            i+=2
        else:
            i+=1     

    return intersection
    ### End your code

简单的测试样例

In [132]:
### Begin your code
list1=[1,3,5,6,7,8,9]
list2=[2,3,4,6,7,10]
for i in sorted_intersect(list1,list2):
    print(i)
### End your code

3
6
7


现在可以利用`sorted_intersect` 和 `InvertedIndexMapper`来实现`retrieve`函数。

In [150]:
#%%tee submission/retrieve.py

class BSBIIndex(BSBIIndex):
    def retrieve(self, query):
        """Retrieves the documents corresponding to the conjunctive query
        
        Parameters
        ----------
        query: str
            Space separated list of query tokens
            
        Result
        ------
        List[str]
            Sorted list of documents which contains each of the query tokens. 
            Should be empty if no documents are found.
        
        Should NOT throw errors for terms not in corpus
        """
        if len(self.term_id_map) == 0 or len(self.doc_id_map) == 0:
            self.load()

        ### Begin your code
        total_docIDs=[]
        #terms-->termID
        for term in query.split():
            if term not in self.term_id_map:
                postings_list=[]
            else:
                termID=self.term_id_map[term]
                #termID-->docIDs
                inverted_index_map=InvertedIndexMapper(self.index_name,self.postings_encoding,directory=self.output_dir)
                inverted_index_map.__enter__()#打开索引文件
                postings_list=inverted_index_map.__getitem__(termID)
            #intersection
            total_docIDs=postings_list if total_docIDs==[] else sorted_intersect(total_docIDs,postings_list)
        #docIDs-->documents
        documents=[]
        for docID in total_docIDs:
            document=self.doc_id_map[docID]
            documents.append(document)

        return documents
        ### End your code



通过一个简单的查询来测试索引在真实数据集上是否正常工作。

In [134]:
BSBI_instance = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir', )
BSBI_instance.retrieve('boolean retrieval')

['1\\cs276.stanford.edu_',
 '1\\cs276a.stanford.edu_',
 '3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 '4\\nlp.stanford.edu_IR-book_',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_putting-it-all-together-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_references-and-furth

你也可以通过读取文件来测试其中的页面是否真的包含了查询的terms

In [135]:
with open("pa1-data/1/cs276.stanford.edu_", 'r') as f:
    print(f.read())

cs276 information retrieval and web search cs 276 ling 286 information retrieval and web search spring 2011 pandu nayak and prabhakar raghavan lecture 3 units tu th 4 15 5 30 gates b03 tas sonali aggarwal ivan cui valentin spitkovsky and sandeep sripada staff e mail cs276 spr1011 staff lists stanford edu lectures are also available online and on television through scpd sitn course description basic and advanced techniques for text based information systems efficient text indexing boolean and vector space retrieval models evaluation and interface issues web search including crawling link based algorithms and web metadata text web clustering classification text mining syllabus additional information staff contact information piazzza we strongly recommend students to post questions to the course page on www piazzza com instead of sending emails this forum enables students to discuss the questions they encounter in lectures or assignments here is a quick introduction video email if you hav

测试dev queries（提前构建好的查询与结果）是否正确

In [136]:
#在给的dev_output文件夹中稍作修改，将文件路径分隔符"/""改为"\\""，以匹配之前join生成的模式
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert set(my_results) == set(reference_results), "Results DO NOT match for query: "+query.strip()  
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


如果出现了错误，可以通过以下代码比较输出与正确结果的差异

In [137]:
set(my_results) - set(reference_results)

set()

In [138]:
set(reference_results) - set(my_results)

set()

确保你构建自己的查询来测试所有可能的边界情况，例如数据集中没有出现的terms，或者拖慢合并的频繁出现的terms。

## 报告：
（1）检索：根据给定的term，在索引文档中查找得到对应的posting；

（2）联合检索需要对query中多个term的posting求交集，主要思路为：首先对query中每个term转化为对应的termID,用InvertedIndexMapper类的__getitem__方法查找得到对应的postings；每得到一组postings，与总postings一起调用sorted_intersection方法得到排序求交集后的结果；如此循环直至所有term处理完毕，将得到的docIDs转化为document名称返回。

（3）其中，__getitem__方法的实现主要是借助元数据，找到该termID在index_file中对应的postings的位置和长度，读取并解码返回；当然，它需要能够处理termID不存在的特殊情况；

（4）其中，sorted_intersection方法的实现是先用heapq.merge方法将两个list合并+排序；然后对合并后的list遍历，只保留出现两次的docID,这样能够在线性时间内完成排序求交集的操作。


# 索引压缩  (30%)

在这部分，你将使用可变长字节编码对索引进行压缩。（gap-encoding可选，对gap进行编码）

下面几个Python运算符可能对你有帮助

In [139]:
# Remainder (modulo) operator %

print("10 % 2 = ", 10 % 2)
print("10 % 3 = ", 10 % 3)

# Integer division in Python 3 is done by using two slash signs

print("10 / 3 = ", 10 / 3)
print("10 // 3 = ", 10 // 3)

10 % 2 =  0
10 % 3 =  1
10 / 3 =  3.3333333333333335
10 // 3 =  3


完成下面的`CompressedPostings`类，我们将用它来替换`UncompressedPostings`。关于如何使用gap-encoding和可变长字节编码，你可以参考老师课件或者[Chapter 5](https://nlp.stanford.edu/IR-book/pdf/05comp.pdf)。

In [140]:
class CompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code

    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` using gap encoding with variable byte 
        encoding for each gap
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
            (as produced by `array.tobytes` function)
        """
        ### Begin your code
        #计算gap
        gap_list=[postings_list[0]]
        for i in range(1,len(postings_list)):
            gap_list.append(postings_list[i]-postings_list[i-1])

        #转化为可变长字节编码
        encoded_string=bytearray()#存储结果的字节数组
        for posting in gap_list:
            while True:
                period_value=posting&0x7F #提取低7位
                posting>>=7 #右移7位
                if posting>0: #若未结束，令最高位为1
                    period_value|=0x80

                encoded_string.append(period_value)#添加到结果数组

                if posting==0: #若已结束，跳出循环
                    break

        return bytes(encoded_string)#返回byte类型
        ### End your code

        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docIds)
        """
        ### Begin your code
        gap_list=[] #存放docIDs的列表
        lower_value=0 #存储未读取完的bytes的值
        i=0 #记录此byte位于int的第几个7位
        for byte in encoded_postings_list:#逐字节处理
            if byte & 0x80: #若最高位为1，暂存至lower_value,并继续读取
                lower_value+=(byte&0x7F)<<(7*i) #将低7位转化为int，乘以对应的级别，存入lower_value
                i+=1
            else:#最高位为0，则合并lower_value结果
                lower_value+=byte<<(7*i) #将byte转化为int，乘以对应的级别,存入lower_value
                gap_list.append(lower_value)
                i=0  #清零
                lower_value=0

        #将gap_list转化为postings_list
        postings_list=[gap_list[0]]
        for i in range(1,len(gap_list)):
            postings_list.append(postings_list[-1]+gap_list[i])#docID=上一个docID+gap
        
        return postings_list
        ### End your code


首先，如果你实现了额外的辅助函数，写一些测试样例

In [141]:
### Begin your code

### End your code

我们实现了一个简单的函数来测试你编码的列表是否被正确解码

In [142]:
def test_encode_decode(l):
    e = CompressedPostings.encode(l)
    print("压缩后的字节数：",len(e))
    d = CompressedPostings.decode(e)
    assert d == l
    print(l, e)

写一些测试样例来确保文档列表的压缩和解压正常运行

In [143]:
### Begin your code
#这个测试样例中既有长度小于7bit的，又有大于的，测试全面
postings_list=[14,17,33,50,89,23989,62443]
#使用普通编码，每个int占4个字节，总字节数：
e1=UncompressedPostings.encode(postings_list)
print(e1)
print("未压缩的字节数:",len(e1))

test_encode_decode(postings_list)
### End your code

b'\x0e\x00\x00\x00\x11\x00\x00\x00!\x00\x00\x002\x00\x00\x00Y\x00\x00\x00\xb5]\x00\x00\xeb\xf3\x00\x00'
未压缩的字节数: 28
压缩后的字节数： 11
[14, 17, 33, 50, 89, 23989, 62443] b"\x0e\x03\x10\x11'\xdc\xba\x01\xb6\xac\x02"


现在让我们创建一个新的输出文件夹并使用`CompressedPostings`来构建压缩索引

In [144]:
try: 
    os.mkdir('output_dir_compressed')
except FileExistsError:
    pass

In [145]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.index()

In [151]:
BSBI_instance_compressed = BSBIIndex(data_dir='pa1-data', output_dir = 'output_dir_compressed', postings_encoding=CompressedPostings)
BSBI_instance_compressed.retrieve('boolean retrieval')

['1\\cs276.stanford.edu_',
 '1\\cs276a.stanford.edu_',
 '3\\infolab.stanford.edu_TR_CS-TN-95-21.html',
 '4\\nlp.stanford.edu_IR-book_',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_an-appraisal-of-probabilistic-models-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_bayesian-network-approaches-to-ir-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_boolean-retrieval-2.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_computing-scores-in-a-complete-search-system-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_dictionaries-and-tolerant-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_efficient-scoring-and-ranking-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_inexact-top-k-document-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_probabilistic-information-retrieval-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_putting-it-all-together-1.html',
 '4\\nlp.stanford.edu_IR-book_html_htmledition_references-and-furth

像之前一样，用已构造好的查询语句来测试是否正常运行

In [153]:
for i in range(1, 9):
    with open('dev_queries/query.' + str(i)) as q:
        query = q.read()
        my_results = [os.path.normpath(path) for path in BSBI_instance_compressed.retrieve(query)]
        with open('dev_output/' + str(i) + '.out') as o:
            reference_results = [os.path.normpath(x.strip()) for x in o.readlines()]
            assert set(my_results) == set(reference_results), "Results DO NOT match for query: "+query.strip()
        print("Results match for query:", query.strip())

Results match for query: we are
Results match for query: stanford class
Results match for query: stanford students
Results match for query: very cool
Results match for query: the
Results match for query: a
Results match for query: the the
Results match for query: stanford computer science


## 报告(索引压缩）：
### 1.方法原理

普通编码是直接将int型的docID调用tobytes()转化为字节数组；这样得到的结果是4个字节为一个单位的，对于一些很小的int数值，这样做浪费了一些空间，因此这里采用gap_list可变长编码方式，去掉无用的"0"，节省空间。

该gap_list可变长编码的原理为：（1）对于每个termID对应的postings_list，第一个docID保持原来的值，后面的改为其与前一个值的差，这样数值变小，需要的存储空间也会变小；（2）对于gap_list的每个值，以7个bit为一个单位存储，若数值x小于2^7，则只需要一个字节(8bit)存储，该字节的最高位为0标识结束；若数值x大于或等于2^7，则将x的低7位存储在第一个字节（该字节最高位为1标识未结束），x右移7位后继续提取低7位，如此循环，直至小于2^7为止，该字节最高位用0标识结束。这样一来，编码后的postings不再以4个字节为一个单位进行读取，而只需要通过每个字节最高位是1/0来判断一个数值的读取是否结束。

解码是编码的逆过程：（1）对于编码后的字节流，逐个字节读取，若最高位为0，则直接提取低7位作为一个docID_gap；若最高位为1，则继续向下读取一个字节直至最高位为0，将每个字节的第七位按顺序拼接计算得到真正的docID_gap。（2）对于gap_list，从第二个元素开始，每个将其gap与前一个元素求和，还原得到真正的docID。最终返回解码后的postings_list。

### 2.实验结果
测试可知，该方法运行正确。经过查看，普通编码得到的output_dir文件大小总量为139MB，使用该方法压缩后得到的output_dir_compressed文件大小总量为65.1MB,压缩比约为2，可见效果显著！




# 额外的编码方式 (10%)

通过补充`ECCompressedPostings`的`encode` 和 `decode`方法来实现一种额外的索引压缩方式。在我们课上学的就是**gamma-encoding** 。另外如果大家感兴趣的话也可以了解**Delta Encoding** ，[ALGORITHM SIMPLE-9](https://github.com/manning/CompressionAlgorithms#simple-9) 等。

你应该以多字节（而不是bits）来存储倒排记录表，因为索引的长度和位置都存的是字节信息。

In [154]:
#gamma-encoding
class ECCompressedPostings:
    #If you need any extra helper methods you can add them here 
    ### Begin your code

    ### End your code
    
    @staticmethod
    def encode(postings_list):
        """Encodes `postings_list` 
        
        Parameters
        ----------
        postings_list: List[int]
            The postings list to be encoded
        
        Returns
        -------
        bytes: 
            Bytes reprsentation of the compressed postings list 
        """
        ### Begin your code
        binary_result=""
        for posting in postings_list:
            if posting==0:#令0的gamma编码为0
                binary_result+="00000000"
            else:
                #int-->bits
                binary_string=bin(posting)[2:]
                ##计算offset
                offset=binary_string[1:]
                #计算length
                length="1"*len(binary_string)
                length+='0'
                #拼接length+offset
                gamma_string=length+offset
                #将二进制串拼接到总的结果中
                binary_result+=gamma_string
        #二进制结果串的长度若为8的倍数，直接转化为bytes，否则需要补齐
        #这里统一在binary_result前面加一个byte，值0表示是8的倍数，否则不是，值表示补齐的位数
        k=len(binary_result)%8
        if k!=0:
            binary_result=format(8-k,'08b')+"0"*(8-k)+binary_result
        else:
            binary_result="0"*8+binary_result
        #将bits转化为bytes
        integer_value=int(binary_result,2) #二进制-->十进制-->bytes
        byte_length=(integer_value.bit_length()+7)//8 #计算字节长度
        encoded_string=integer_value.to_bytes(byte_length,'big') #转化为bytes

        return encoded_string
        ### End your code
        
        
    @staticmethod
    def decode(encoded_postings_list):
        """Decodes a byte representation of compressed postings list
        
        Parameters
        ----------
        encoded_postings_list: bytes
            Bytes representation as produced by `CompressedPostings.encode` 
            
        Returns
        -------
        List[int]
            Decoded postings list (each posting is a docId)
        """
        ### Begin your code
        postings_list=[]
        #十六进制串-->二进制串
        binary_string=''.join(format(byte,'08b')for byte in encoded_postings_list)
        
        #先读取前8个bit，判断是否有填充，并将指针i移动到真正的bit开始
        post_8bits=binary_string[:8]
        t=int(post_8bits,2)
        if t>0:
            i=8+t
        else:
            i=8
        #遍历后续的bit，进行解码
        l=0 #l记录length
        while i <len(binary_string):
            #获取length
            if binary_string[i]=='1':
                while i<len(binary_string) and binary_string[i]=='1':
                    l+=1#记录1的个数，即为长度
                    i+=1
                i+=1#跳过length后面的'0'
                binary_posting="1"#最高位是1
                for j in range(i,i+(l-1)):#读取后面的offset
                    binary_posting+=binary_string[j]
                postings_list.append(int(binary_posting,2))#转化为十进制，添加到结果中
                i+=(l-1)#跳过offset
                l=0#重置length
            else:#是0
                postings_list.append(0)
                i+=8 #跳过8位，即一个字节0
        return postings_list
        ### End your code

In [155]:
#测试gamma-encoding的正确性
postings_list=[14,17,33,50,89,23989,62443]

#使用普通编码，每个int占4个字节，总字节数：
e1=UncompressedPostings.encode(postings_list)
print(e1)
print("未压缩的字节数:",len(e1))

#使用gamma编码，总字节数
e2=ECCompressedPostings.encode(postings_list)
print(e2)
print("压缩后的字节数:",len(e2))
d=ECCompressedPostings.decode(e2)
print(d)
assert d==postings_list,"error"

b'\x0e\x00\x00\x00\x11\x00\x00\x00!\x00\x00\x002\x00\x00\x00Y\x00\x00\x00\xb5]\x00\x00\xeb\xf3\x00\x00'
未压缩的字节数: 28
b'\x02=\xbe\x1f\xc1\xfd/\xe6\x7f\xff\x9d\xb5\xff\xffs\xeb'
压缩后的字节数: 16
[14, 17, 33, 50, 89, 23989, 62443]


## 报告（额外编码方式）：
### 1.方法原理
使用gamma-encoding编码方法，它也是一种可变长编码方法，原理为：将一个数x的组成分成length和offset两部分，其中length部分用来标识x的bit位数，有几位则length部分就是几个"1"连接而成，最后加一个"0"表示结束；offset部分用来标识x的偏移量，取原本数值n位的低n-1位（因为第n位一定是1）。length和offset拼接得到编码后的二进制表示；

解码时，先通过记录连续的"1"的个数来获知数x的bit位数（假设为l），然后对于后面的l-1位，读取出来即为offset。最终结果是在offset最高位的高一位添加"1"还原出本来的数值，再转化为十进制的posting。

### 2.细节处理

由于gamma-encoding编码原理主要基于二进制特性，在实际的索引存储中需要转换为16进制的字节流；因此，我对每个termID对应的postings_list转化得到的二进制串，使用如下方法补齐位数，使其能刚好转化为字节串：

设原本编码后的二进制串用s表示，若s恰好为8k个bits，那么在其前面加一个字节0(8个bits的0)，来表示可以直接读取； 若s的长度不足8k个bits，设其至少添加m个前导0补足8k个bits，则再在前面加一个字节，值大小为m，表示要从m位往后开始读取。这样，就不会因为二进制转化为字节流的位数不匹配问题而导致解码失败。

### 3.实验结果
由上面的测试可知，普通方法需要28个字节存储的list，使用该编码方法只需要16个字节，大大压缩了存储空间。